In [ ]:
!pip install pubchempy

In [24]:
import pubchempy as pcp
import pandas as pd
import numpy as np
from tqdm import tqdm
# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [71]:
# в нашем датасете много повторяющихся SMILES, для ускорения процесса будем искать дескрипторы только для уникальных SMILES - они в df_short
df_start = pd.read_csv('/content/df_9_after_cleansing.csv')
df_short = df_start.groupby(['SMILES'], as_index = False).mean()


In [5]:
pubchempy_list = ['MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IsomericSMILES', 'InChI', 'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 'EffectiveRotorCount3D', 'ConformerCount3D']
pcp.get_properties(pubchempy_list, 'CCC', 'smiles', as_dataframe=True)


,MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,InChI,InChIKey,IUPACName,XLogP,ExactMass,MonoisotopicMass,...,FeatureCount3D,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D
CID,,,,,,,,,,,,,,,,,,,,,
6334,C3H8,44.10,CCC,CCC,"InChI=1S/C3H8/c1-3-2/h3H2,1-2H3",ATUOYWHBWRKTHZ-UHFFFAOYSA-N,propane,1.8,44.062600255,44.062600255,...,1,0,0,0,0,0,1,0.4,0,1


In [31]:
#20 min
df_pubchempy = pd.DataFrame(columns = pubchempy_list)
for i, _ in tqdm(df_short.iterrows()):
  try:
    df_pubchempy = pd.concat([df_pubchempy, pcp.get_properties(pubchempy_list, df_short['SMILES'][i], 'smiles', as_dataframe=True)])
  except:
    df_pubchempy = pd.concat([df_pubchempy, pd.DataFrame(np.nan, columns = pubchempy_list, index = [df_short['SMILES'][i]])])

865it [19:42,  1.37s/it]


In [35]:
df_pcp = df_pubchempy.reset_index().rename(columns = {'index':'CID'})
df_pcp['CanonicalSMILES'] = df_pcp['CanonicalSMILES'].combine_first(df_pcp['CID'])
df_pcp = df_pcp.set_index('CanonicalSMILES')

In [37]:
df_pcp.to_csv('dict_for_pcp_descriptors.csv')

In [91]:
df_pcp = pd.read_csv('dict_for_pcp_descriptors.csv')
df_padel = pd.read_csv('dict_for_padel_descriptors.csv', index_col = 'Unnamed: 0')
df_rdkit = pd.read_csv('df_rdkit.csv', index_col = 'Unnamed: 0.1')

In [95]:
df_new_pcp = pd.DataFrame(columns = df_pcp.columns)
for i, _ in tqdm(df_start.iterrows()):
  df_new_pcp = pd.concat([df_new_pcp, df_pcp.iloc[df_short[df_short['SMILES'] == df_start['SMILES'][i]].index]])

2973it [00:05, 543.23it/s]


In [96]:
df_new_padel = pd.DataFrame(columns = df_padel.columns)
for i, _ in tqdm(df_start.iterrows()):
  df_new_padel = pd.concat([df_new_padel, df_padel.iloc[df_short[df_short['SMILES'] == df_start['SMILES'][i]].index]])
    

2973it [00:27, 108.97it/s]


In [97]:
df_new_pcp_1 = df_new_pcp.reset_index(drop = True)
df_new_padel_1 = df_new_padel.reset_index(drop = True)
df_new_rdkit = df_rdkit.iloc[:, 12:]

In [98]:
df_final_with_descriptors = pd.concat([df_start, df_new_rdkit, df_new_pcp_1, df_new_padel_1], axis = 1).drop(['Unnamed: 0'], axis = 1)

In [100]:
df_final_with_descriptors.to_csv('df_final_with_descriptors.csv')

In [99]:
df_final_with_descriptors

,Date,Journal,Title,Name,wavelength_added,RI,DOI_alt,SMILES,Method_ord,Publisher_ord,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,5/25/2011,Physical Chemistry Chemical Physics,Kirkwood correlation factors in liquid mixture...,Triglyme,589.0,1.421120,10.1039/C1CP20142E,COCCOCCOCCOC,1,1,...,0.946528,0.031934,0.549275,0.526084,0.492699,19.717294,19.944032,44.651635,0.919793,1.568058
1,9/13/1999,Journal of Alloys and Compounds,GROWTHOPTICALPROPERTIESREDOPEDBULKFIBERSINGLEC...,YVO4,589.0,2.168000,10.1016/S0925-8388(98)00420-4,[O-2].[O-2].[O-2].[O-2].[V].[Y+3],1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/4/1999,Colloids and Surfaces A: Physicochemical and E...,ADSORPTIONOILSURFACTANTMONOLAYERSSTRUCTUREMIXE...,Toluene,589.0,1.494000,10.1016/S0927-7757(98)00782-1,CC1=CC=CC=C1,1,0,...,0.645928,0.332469,0.550605,0.453388,0.125625,5.009615,5.919881,11.512744,0.468892,1.129618
3,8/5/2004,Materials Science and Engineering: B,Fabrication of PECVD-silicon oxynitride-based ...,silicon dioxide,589.0,1.460000,10.1016/j.mseb.2004.05.037,O=[Si]=O,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/1/2016,Optik,OPTIMIZATIONENHANCEMENTLIQUIDANALYTESENSINGPER...,Benzyne,589.0,1.354000,10.1016/j.ijleo.2016.11.171,C1=CC#CC=C1,2,0,...,0.560564,0.426347,0.464710,0.524683,0.276782,3.839450,3.713540,7.730048,0.480366,1.266175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,7/30/2011,Thermochimica Acta,THERMOPHYSICALSTUDYNHEXANENHEPTANE1CHLOROPROPA...,n-Heptane,589.0,1.390333,10.1016/j.tca.2011.07.023,CCCCCCC,0,0,...,0.856853,0.086350,0.562047,0.557496,0.590060,8.745269,9.755805,21.311787,0.785279,1.709602
2969,6/9/2015,RSC Adv.,Semitransparent polymer-based solar cells via ...,TiO2,26.0,1.640000,10.1039/C5RA05762K,O=[Ti]=O,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2970,6/17/2015,Phys. Chem. Chem. Phys.,Transformation of photophysical properties fro...,"1,4-Dioxane",589.0,1.420300,10.1039/C5CP02762D,C1COCCO1,0,1,...,0.552766,0.330474,0.649745,0.843263,0.503156,4.024970,4.630103,10.045862,0.329148,1.996164
2971,4/13/2017,Vibrational Spectroscopy,IMPROVEMENTSTIO2CATALYZEDAGNPSBASEDSERSSUBSTRA...,TiO2,589.0,1.630000,10.1016/j.vibspec.2017.03.005,O=[Ti]=O,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
